# CICE-MOM6 

We'll go through these steps:

1. Generate a regional MOM6 domain.
2. Create the CESM case.
3. Prepare ocean forcing data.
4. Generate the regional CICE grid.
5. Build and run the case.

Remember to swap your environment to CrocoDash first!

# Section 1: Generate a regional MOM6 domain



## Step 1.1: Horizontal Grid

CROCODILE grid generator tools cannot create domains over the poles (but will soon!), instead we pass in previously generated grids over the Artic/Antartic (If you're interested in how we generated these, please reach out!). As a quick reminder, sea ice OBCs are not well understood, and it is common practice to have a domain where sea ice does not reach the boundaries.

In [ ]:
from CrocoDash.grid import Grid

grid = Grid.from_supergrid("")
grid.name = "Arctic_Grid"

## Step 1.2: Topography

In [ ]:
from CrocoDash.topo import Topo

bathymetry_path=''

topo = Topo.from_topo_file(
    grid = grid,
    topo_file_path=bathymetry_path,
    min_depth = 9.5,
)

In [ ]:
topo.depth.plot()

Run the topo-editor below to configure your bathymetry!

In [ ]:
%matplotlib ipympl
from CrocoDash.topo_editor import TopoEditor
topo.depth["units"] = "m"
TopoEditor(topo)

## Step 1.3: Vertical Grid

In [ ]:
from CrocoDash.vgrid import VGrid

vgrid  = VGrid.hyperbolic(
    nk = 75,
    depth = topo.max_depth,
    ratio=20.0
)

In [ ]:
print(vgrid.dz)

In [ ]:
import matplotlib.pyplot as plt
plt.close()
# Create the plot
for depth in vgrid.z:
    plt.axhline(y=depth, linestyle='-')  # Horizontal lines

plt.ylim(max(vgrid.z) + 10, min(vgrid.z) - 10)  # Invert y-axis so deeper values go down
plt.ylabel("Depth")
plt.title("Vertical Grid")
plt.show()

# SECTION 3: Create the CESM case

After generating the MOM6 domain, the next step is to create a CESM case using CrocoDash. This process is straightforward and involves instantiating the CrocoDash Case object. The Case object requires the following inputs:

 - CESM Source Directory: A local path to a compatible CESM source copy.
 - Case Name: A unique name for the CESM case.
 - Input Directory: The directory where all necessary input files will be written.
 - MOM6 Domain Objects: The horizontal grid, topography, and vertical grid created in the previous section.
 - Project ID: (Optional) A project ID, if required by the machine.

## Step 2.1: Specify case name and directories:

Begin by specifying the case name and the necessary directory paths. Ensure the CESM root directory points to your own local copy of CESM. 
**Note that if the case is already created - in case you want to rerun this notebook - the `/run` directory must be removed for the case to be created anew.**
Below is an example setup:

In [ ]:
from pathlib import Path

In [ ]:
# CESM case (experiment) name
casename = "arctic_cice"

# CESM source root (Update this path accordingly!!!)
cesmroot = "<CESM>"

# Place where all your input files go
inputdir = Path.home() / "input" / casename

# CESM case directory
caseroot = Path.home() / "cases" / casename


## Step 2.2: Create the Case


In [ ]:
from CrocoDash.case import Case

case = Case(
    cesmroot = cesmroot,
    caseroot = caseroot,
    inputdir = inputdir,
    ocn_grid = grid,
    ocn_vgrid = vgrid,
    ocn_topo = topo,
    project = 'P93300012', # Switch this
    override = True,
    machine = "derecho", # And this
    compset = "1850_DATM%JRA_SLND_CICE_MOM6_SROF_SGLC_SWAV",  # See how we selected CICE? That's the only step you need additional for CICE-MOM6 coupling!
)

# Section 3: Prepare ocean forcing data


## Step 3.1 Configure Initial Conditions and Forcings

In [ ]:
from CrocoDash.case import Case

# Add whatever forcings you see fit!
case.configure_forcings(
    date_range = ["START (format %Y-%m-%d 00:00:00)", "END (format %Y-%m-%d 00:00:00)"],
)

## Step 3.2 Run `get_glorys_data.sh`



##  Step 3.3: Process forcing data


In [ ]:
case.process_forcings()

## Optional: Use restart files as initial condition for your run (after you run for the first time!)

In case you want to start the model with a restart file instead of using the generated initial condition, follow the below steps. Note that you have to have finished a run beforehand for the restart files to appear.
1. Locate your restart file - they are usually in your previous case's `/archive/rest/<year>` folder with a `.r` infix, e.g., `cice.test.cice.r.1994-01-01-00000.nc`.
   If you do not know your `/archive` folder location, run `./xmlquery DOUT_S_ROOT` in your (previous) case folder, which will return a path similar to `DOUT_S_ROOT: <PATH TO ARCHIVE>`.

2. Use the `cp` command to copy the file to your current case `/run` directory, e.g., `cp cice.test.cice.r.1994-01-01-00000.nc <your_run_dir>`.
3. Open `user_nl_cice` in your case directory again and change the `ice_ic` variable from `"UNSET"` to your file name, e.g., `ice_ic=cice.test.cice.r.1994-01-01-00000.nc`

Your file will now be used automatically as the ice initial condition for your next run. Note that `.h` and `.h1` files (i.e. history files) currently **cannot be used** as initial conditions for CICE.

# Section 5: Build and run the case

After completing the previous steps, you are ready to build and run your CESM case. Begin by navigating to the case root directory specified during the case creation. Before proceeding, review the `user_nl_mom` file located in the case directory. This file contains MOM6 parameter settings that were automatically generated by CrocoDash. Carefully examine these parameters and make any necessary adjustments to fine-tune the model for your specific requirements. While CrocoDash aims to provide a solid starting point, further tuning and adjustments are typically necessary to improve the model for your use case.

Once you have reviewed and modified the parameters as needed, you can build and execute the case using the following commands: 
```
./case.build
./case.submit
```